# 🧪 Laboratório 2 — **Paralelismo** com Multiagentes: Monitor de Mercado Multi-Ativo (CrewAI)

**Objetivo do Lab:** aprender a projetar **tarefas paralelas (fan-out)** e consolidar resultados com um agente **agregador (fan-in)** para produzir um relatório educacional sobre **Ações (MSFT)**, **Câmbio (USDBRL=X)** e **Ação brasileira (PETR4.SA)**.

> Ritmo: **conceito → exemplo → código guiado → execução → variações**  
> Foco: **Process.parallel** do CrewAI + uso de **yfinance** para dados reais.


## 🔑 LLM (Groq) — configuração rápida (OpenAI-compatible)

1. Crie sua conta e gere uma chave em: https://console.groq.com  
2. No Colab, defina **variáveis de ambiente** do provedor compatível:
   - `OPENAI_API_KEY`: sua chave da Groq  
   - `OPENAI_API_BASE`: endpoint → `https://api.groq.com/openai/v1`

> Com isso, os agentes/tarefas do CrewAI usarão a Groq automaticamente.


## 🔧 Passo 0 — Instalar dependências
Execute a célula abaixo. Se o Colab pedir para reiniciar, aceite e depois rode novamente esta célula.


In [ ]:
!pip -q install crewai crewai-tools langchain python-dotenv yfinance pandas numpy


## 🔐 Passo 1 — Configurar chaves no ambiente (Groq ou outro provedor compatível)

> Opcionalmente, cole suas chaves abaixo (somente para este notebook).  
> Em produção, prefira **segredos** e **variáveis de ambiente** seguras.


In [ ]:
import os

# ===> DESCOMENTE e COLE suas chaves para usar a Groq (OpenAI-compatible) <===
# os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_DA_GROQ_AQUI"
# os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

print("OPENAI_API_KEY set?", "OPENAI_API_KEY" in os.environ)
print("OPENAI_API_BASE set?", "OPENAI_API_BASE" in os.environ)

## 🧠 Passo 2 — Conceitos rápidos: **paralelo vs sequencial**

- **Sequencial**: T1 → T2 → T3 (um espera o outro terminar).  
- **Paralelo**: T1, T2, T3 **ao mesmo tempo** (quando independentes).  
- **Fan-out/Fan-in**: várias tarefas em paralelo (fan-out) → **agregador** consolida (fan-in).

No CrewAI, usamos `Process.parallel` para disparar tarefas independentes simultaneamente.


## ✅ Passo 3 — Imports e checagem


In [ ]:
from crewai import Agent, Task, Crew, Process
import pandas as pd, numpy as np
import datetime as dt

print("CrewAI importado com sucesso.")


## 📥 Passo 4 — Captura de dados de mercado (yfinance)

Vamos buscar **últimos ~90 dias** para três ativos:  
- `MSFT` (Ações EUA)  
- `USDBRL=X` (Câmbio Dólar/Real)  
- `PETR4.SA` (Ação brasileira)


In [ ]:
import yfinance as yf

TICKERS = {
    "acoes_msft": "MSFT",
    "fx_usdbrl": "USDBRL=X",
    "acoes_petr4": "PETR4.SA",
}

def fetch_last_90d_prices(ticker):
    end = dt.date.today()
    start = end - dt.timedelta(days=120)
    data = yf.download(ticker, start=start, end=end, progress=False)
    if data is None or data.empty:
        raise RuntimeError(f"Sem dados para {ticker}.")
    df = data.reset_index()[["Date","Close"]].rename(columns={"Date":"date","Close":"close"})
    df["date"] = df["date"].astype(str)
    return df.tail(90).reset_index(drop=True)

raw_data = {}
for k, t in TICKERS.items():
    try:
        raw_data[k] = fetch_last_90d_prices(t)
    except Exception as e:
        print("Falha ao obter", t, ":", e)
        raw_data[k] = pd.DataFrame({"date": [], "close": []})

{key: df.tail(3) for key, df in raw_data.items()}


## 🔢 Passo 5 — Cálculo de métricas (simples)
Vamos derivar **médias móveis** e **variações** de curto prazo para cada série.


In [ ]:
def compute_trend_metrics(df: pd.DataFrame, win_short=5, win_long=20):
    if df is None or df.empty:
        return pd.DataFrame()
    dfx = df.copy()
    dfx["ma_short"] = dfx["close"].rolling(win_short).mean()
    dfx["ma_long"]  = dfx["close"].rolling(win_long).mean()
    dfx["pct_5"]    = dfx["close"].pct_change(5)
    dfx["pct_20"]   = dfx["close"].pct_change(20)
    return dfx

trend_data = {k: compute_trend_metrics(df) for k, df in raw_data.items()}
{key: df.tail(3) for key, df in trend_data.items()}


## 🧩 Passo 6 — Definir Agentes (papéis)

- **Analista de Ações (MSFT)**  
- **Analista de Câmbio (USDBRL)**  
- **Analista de Ação BR (PETR4)**  
- **Redator Educacional (agregador)


In [ ]:
# Especialistas (executam em paralelo)
analyst_equity_us = Agent(
    role="Analista de Ações EUA (MSFT)",
    goal="Gerar um panorama educacional claro do comportamento recente de MSFT",
    backstory="Você é um analista de equities focado em leitura objetiva de curto prazo.",
    allow_delegation=False,
    verbose=True
)

analyst_fx_brl = Agent(
    role="Analista de Câmbio (USD/BRL)",
    goal="Explicar a dinâmica recente do câmbio USD/BRL de modo didático",
    backstory="Você acompanha macroeconomia e moedas, com foco no real.",
    allow_delegation=False,
    verbose=True
)

analyst_equity_br = Agent(
    role="Analista de Ações BR (PETR4)",
    goal="Gerar um panorama educacional claro do comportamento recente de PETR4",
    backstory="Você analisa ações brasileiras com olhar em fatores locais e globais.",
    allow_delegation=False,
    verbose=True
)

# Agregador (fan-in)
educational_writer = Agent(
    role="Redator Educacional",
    goal=("Consolidar as análises em um relatório didático, conectando conceitos "
          "e destacando relações entre os mercados."),
    backstory=("Você é um instrutor que transforma análises técnicas em conteúdo "
               "acessível para estudantes e iniciantes."),
    allow_delegation=False,
    verbose=True
)


## 🗂️ Passo 7 — Preparar **contexto** textual para cada agente

Para economizar tokens e orientar a análise, passaremos **apenas o recorte final** (últimos 10 pontos) com métricas.


In [ ]:
def tail_context(df: pd.DataFrame, cols):
    if df is None or df.empty:
        return "Sem dados disponíveis."
    return df.tail(10)[cols].to_string(index=False)

ctx_msft  = tail_context(trend_data["acoes_msft"], ["date","close","ma_short","ma_long","pct_5","pct_20"])
ctx_usdbrl= tail_context(trend_data["fx_usdbrl"], ["date","close","ma_short","ma_long","pct_5","pct_20"])
ctx_petr4 = tail_context(trend_data["acoes_petr4"], ["date","close","ma_short","ma_long","pct_5","pct_20"])

print("Prévia do contexto (MSFT):\n", ctx_msft.splitlines()[-5:])


## ⚙️ Passo 8 — Definir **Tarefas em Paralelo** (fan-out)

Cada tarefa recebe **somente** seu contexto. Depois, faremos o **fan-in** no Redator.


In [ ]:
task_msft = Task(
    description=(
        "Explique de forma **educacional** o comportamento recente de MSFT usando os dados abaixo. "
        "Indique se a leitura sugere tendência de **alta/baixa/lateral** (curto prazo) considerando "
        "médias móveis e variações. Evite jargões e use até 6 linhas.\n\n"
        f"--- DADOS MSFT ---\n{ctx_msft}\n--- FIM ---"
    ),
    expected_output="Um parágrafo didático sobre MSFT (até 6 linhas).",
    agent=analyst_equity_us,
)

task_usdbrl = Task(
    description=(
        "Explique de forma **educacional** o comportamento recente do câmbio USD/BRL usando os dados abaixo. "
        "Indique se a leitura sugere tendência de **alta/baixa/lateral** (curto prazo). Evite jargões e use até 6 linhas.\n\n"
        f"--- DADOS USD/BRL ---\n{ctx_usdbrl}\n--- FIM ---"
    ),
    expected_output="Um parágrafo didático sobre USD/BRL (até 6 linhas).",
    agent=analyst_fx_brl,
)

task_petr4 = Task(
    description=(
        "Explique de forma **educacional** o comportamento recente de PETR4 usando os dados abaixo. "
        "Indique se a leitura sugere tendência de **alta/baixa/lateral** (curto prazo). Evite jargões e use até 6 linhas.\n\n"
        f"--- DADOS PETR4 ---\n{ctx_petr4}\n--- FIM ---"
    ),
    expected_output="Um parágrafo didático sobre PETR4 (até 6 linhas).",
    agent=analyst_equity_br,
)


## 🧲 Passo 9 — **Agregador** (fan-in): consolidar tudo em relatório educacional

O Redator recebe o **contexto** das três análises e escreve um **texto único** com introdução, síntese dos três mercados e **conexões** entre eles.


In [ ]:
task_report = Task(
    description=(
        "Consolide as análises dos três especialistas em um **relatório educacional** (Markdown) com:\n"
        "1) Introdução (1 parágrafo): objetivo do monitor.\n"
        "2) Sínteses breves: MSFT, USD/BRL, PETR4 (1 parágrafo cada).\n"
        "3) Conexões e aprendizados (1 parágrafo): como os mercados se influenciam.\n"
        "4) Conclusão (1 parágrafo): principais sinais a observar no curto prazo.\n"
        "Evite jargões, seja didático e **não invente dados** que não foram passados."
    ),
    expected_output="Relatório final em Markdown, com 5 parágrafos (educacional).",
    agent=educational_writer,
    context=[task_msft, task_usdbrl, task_petr4],
)


## 🚀 Passo 10 — Montar o **Crew** e executar em **paralelo**

Repare no `process=Process.hierarchical` para disparar as três análises ao mesmo tempo.


In [ ]:
crew_parallel = Crew(
    agents=[analyst_equity_us, analyst_fx_brl, analyst_equity_br, educational_writer],
    tasks=[task_msft, task_usdbrl, task_petr4, task_report],
    process=Process.hierarchical,  # 🔑 fan-out das três análises
    manager_llm="gpt-3.5-turbo",
    verbose=True
)

print(">>> Executando Lab 2 (paralelo)...")
result_parallel = crew_parallel.kickoff()

## 👀 Passo 11 — Visualizar o relatório final (Markdown)


In [ ]:
from IPython.display import display, Markdown
display(Markdown(result_parallel.raw))


## 🔁 Desafios e variações (para praticar)
1. **Experimento de tempo**: mude para `Process.sequential` e compare o tempo vs paralelo.
2. **Novo agente**: adicione um **Analista de Commodities (Brent)** com `BZ=F` e inclua no fan-in.
3. **Pesquisa de notícias**: injete um resumo de manchetes recentes (manual ou via Serper/yfinance) no contexto do agregador.
4. **Camada de validação**: crie um agente “Revisor de Consistência” que critique o relatório antes de publicar.
